In [1]:
# pip install https://github.com/googleapis/python-analytics-data
from apiclient.discovery import build 
from oauth2client.service_account import ServiceAccountCredentials
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimensionimport plotly.express as px
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import pandas as pd

In [2]:
# https://lvngd.com/blog/access-the-google-analytics-reporting-api-with-python/

In [3]:
# VIEW ID Gives the analytics tracker you want to connect to (e.g. Wiki, robots, ros.org)
# You'll need one for each property
# Example: https://keyword-hero.com/documentation/finding-your-view-id-in-google-analytics
# Example: https://analytics.google.com/analytics/web/#/a17821189w35999412p183497681/admin/view/settings
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './ganalyticstest-254023-93ec92d191d6.json'
WIKI_VIEW_ID = '76364704'
DISCOURSE_VIEW_ID = '117310724'
ANSWERS_VIEW_ID = '42382471'
ROSCON_VIEW_ID = '52532925'
ROSORG_VIEW_ID = '79652481'
ROBOTS_VIEW_ID = '129454749'
ROS2_VIEW_ID = '87843515'
INDEX_VIEW_ID = '183497681'

websites = [WIKI_VIEW_ID,DISCOURSE_VIEW_ID,ANSWERS_VIEW_ID,ROSORG_VIEW_ID,ROBOTS_VIEW_ID,INDEX_VIEW_ID]
web_names = ["ROS Wiki","ROS Discourse","ROS Answers","ROS.org","robots.ros.org","ROS Index"]

In [4]:
credentials = ServiceAccountCredentials.from_json_keyfile_name( 
            KEY_FILE_LOCATION, SCOPES)

In [5]:
analytics = build('analyticsreporting', 'v4', credentials=credentials)

In [6]:
# Note, you must grep the keyfile for the e-mail address, 
# e.g. blah@foo.iam.gserviceaccount.com and, and then add it
# to the authorized user account.
# i.e. https://analytics.google.com/analytics/web/#/a17821189w35999412p76364704/admin/suiteusermanagement/account
# See: https://stackoverflow.com/questions/12837748/analytics-google-api-error-403-user-does-not-have-any-google-analytics-account

In [7]:
# API REFERENCE: https://developers.google.com/analytics/devguides/reporting/core/v3/reference#metrics
# Metrics REFERENCE: https://ga-dev-tools.web.app/dimensions-metrics-explorer/

In [42]:
response = analytics.reports().batchGet( 
    body={ 
        'reportRequests': [ 
            { 
                'viewId': ROSCON_VIEW_ID, 
                'dateRanges': [{'startDate': '2021-07-14', 'endDate': '2021-07-14'},
                               {'startDate': '2020-07-14', 'endDate': '2020-07-14'}], 
                'metrics': [{'expression': 'ga:hits'}] 
                }] 
                } 
            ).execute()

In [43]:
response

{'reports': [{'columnHeader': {'metricHeader': {'metricHeaderEntries': [{'name': 'ga:hits',
       'type': 'INTEGER'}]}},
   'data': {'isDataGolden': True,
    'maximums': [{'values': ['257']}, {'values': ['88']}],
    'minimums': [{'values': ['257']}, {'values': ['88']}],
    'rowCount': 1,
    'rows': [{'metrics': [{'values': ['257']}, {'values': ['88']}]}],
    'totals': [{'values': ['257']}, {'values': ['88']}]}}]}

In [44]:
metrics = [{'expression': 'ga:hits'}]
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021]
viewid = WIKI_VIEW_ID
def fetch_stats( viewid, metrics, years, start = "{0}-07-01", stop = "{0}-07-07"):
    results = []
    for idx in range(0,len(years)):
        start_date = start.format(years[idx])
        stop_date = stop.format(years[idx])    
        response = analytics.reports().batchGet( 
            body={ 
                'reportRequests': [ 
                    { 
                        'viewId': viewid, 
                        'dateRanges': [{'startDate': start_date, 'endDate': stop_date}], 
                        'metrics': metrics  
                        }] 
                    }).execute()
        results.append(response["reports"][0]["data"]["totals"][0]["values"][0])
    return results
    
#print(fetch_stats(WIKI_VIEW_ID,metrics,years))

In [45]:
hit_stats = []
for web in websites:
    hit_stats.append(fetch_stats(web,metrics,years))


In [46]:
for name,stat in zip(web_names,hit_stats):
    print(name,stat)

ROS Wiki ['0', '218840', '267810', '295702', '414117', '486583', '521988', '460224', '456847']
ROS Discourse ['0', '0', '0', '1218', '5428', '8808', '12947', '18988', '19329']
ROS Answers ['47065', '81665', '66948', '93578', '118528', '143152', '168471', '173500', '191410']
ROS.org ['0', '30760', '55555', '65575', '67458', '72083', '77019', '60026', '60371']
robots.ros.org ['0', '0', '0', '0', '5573', '5506', '6177', '6325', '5134']
ROS Index ['0', '0', '0', '0', '0', '0', '26019', '57177', '9495']


In [48]:
# Rejigger our dataset to have values underr a column
proto_df = []
for idx, year in enumerate(years):
    temp = {}
    temp["year"] = year
    for name,stat in zip(web_names,hit_stats):
        temp[name] = int(stat[idx])
    proto_df.append(temp)

print(proto_df)
df = pd.DataFrame(data=proto_df)

df.head()

[{'year': 2013, 'ROS Wiki': 0, 'ROS Discourse': 0, 'ROS Answers': 47065, 'ROS.org': 0, 'robots.ros.org': 0, 'ROS Index': 0}, {'year': 2014, 'ROS Wiki': 218840, 'ROS Discourse': 0, 'ROS Answers': 81665, 'ROS.org': 30760, 'robots.ros.org': 0, 'ROS Index': 0}, {'year': 2015, 'ROS Wiki': 267810, 'ROS Discourse': 0, 'ROS Answers': 66948, 'ROS.org': 55555, 'robots.ros.org': 0, 'ROS Index': 0}, {'year': 2016, 'ROS Wiki': 295702, 'ROS Discourse': 1218, 'ROS Answers': 93578, 'ROS.org': 65575, 'robots.ros.org': 0, 'ROS Index': 0}, {'year': 2017, 'ROS Wiki': 414117, 'ROS Discourse': 5428, 'ROS Answers': 118528, 'ROS.org': 67458, 'robots.ros.org': 5573, 'ROS Index': 0}, {'year': 2018, 'ROS Wiki': 486583, 'ROS Discourse': 8808, 'ROS Answers': 143152, 'ROS.org': 72083, 'robots.ros.org': 5506, 'ROS Index': 0}, {'year': 2019, 'ROS Wiki': 521988, 'ROS Discourse': 12947, 'ROS Answers': 168471, 'ROS.org': 77019, 'robots.ros.org': 6177, 'ROS Index': 26019}, {'year': 2020, 'ROS Wiki': 460224, 'ROS Discours

,year,ROS Wiki,ROS Discourse,ROS Answers,ROS.org,robots.ros.org,ROS Index
0,2013,0,0,47065,0,0,0
1,2014,218840,0,81665,30760,0,0
2,2015,267810,0,66948,55555,0,0
3,2016,295702,1218,93578,65575,0,0
4,2017,414117,5428,118528,67458,5573,0


In [49]:
fig = px.line(df, x="year", y=df.columns)
fig.show()
px.line?